# Weld Demo

In this notebook, we will explore implementing a very simple vector library similar to NumPy using Weld. Weld is
a API and runtime for accelerating data parallel computations. Weld is particularly useful when computations are composed of smaller functions in a library.

## NumPy Example

Let's start with a simple NumPy example. We will write a function which adds an integer to a vector many times. Seems pretty simple! 

In [1]:
import numpy as np
import time

import os
os.environ["WELD_NUM_THREADS"] = "1"

In [2]:
# Global constants
SIZE = (2 << 28)
ITERATIONS = 40

In [3]:
def run_numpy_addition(a):
    for i in xrange(ITERATIONS):
        a += i
    print "Result:", a.sum()

In [4]:
a = np.zeros((SIZE), dtype='int32')
run_numpy_addition(a)

Result: 418759311360


As you can see, that took a rather long time to return a result. Why is that? Most libraries have highly optimized
_individual functions_; composing these fast operators in a larger pipeline can often lead to surprisingly long execution times!

The above program is slow because significant time is spent scanning through `a` and adding a single number
to every element in the array. In other words, the CPU is actually spending most of its time doing memory I/O as opposed to actual arithmetic operations.

This is where Weld comes in! By looking at the entire computation at once, Weld can optimize across _all the operations in a program_ rather than each _individual operation_. In workloads like the above, where
the operations themselves are not particularly compute-heavy, these kinds of optimizations become very important.

### Implementing a Weld-enabled Vector Library

We will start this notebook by implementing our very own vector library with Weld. Our library will only support integers and two operations: _summing a vector_ and _adding an integer to each element in a vector_. This should be enough for us to "Weld-ify" the above example!

In [5]:
from weld.weldobject import *
from weld.types import *

## `HelloWeldVector`

We will call our class `HelloWeldVector`, which will wrap a NumPy array (we will use NumPy to manage memory) and build up a Weld computation:

In [6]:
class HelloWeldVector(object):
    pass

### Initialization

Now, let's define some methods on this class. First, `__init__` to create an instance. A few things to note here:

1. We take as input an initial vector (this will be a NumPy array with `dtype="int32"`)
2. We initialize a `WeldObject`, which manages the data passed into the Weld runtime as well as the computation that will be eventually executed. More on the `_encoder` and `_decoder` later.
3. We get a `name` for the current state. Names are basically strings which identify a if a computation currently exists. The current computation is just a single vector of integers; the `weldobj.update` function registers this fact and returns a name.
4. Set the code to `name`; the `weld_code` is the actual program Weld executes. Again, this is just returning the input vector for now.

That was a lot of stuff! But we're making great progress!

In [151]:
def __init__(self, vector):
    self.vector = vector
    self.weldobj = WeldObject(_encoder, _decoder)
    name = self.weldobj.update(vector, WeldVec(WeldInt()))
    self.weldobj.weld_code = name

### Elementwise `add` operator

We will now add an `add` operator. This takes our vector and adds the passed in `number` to each element. The Weld Intermediate Language looks similar to a typical functional programming language where we might do something like the following to add a number `n` to each element of a vector `v`:

```
map(v, |x| x + n)
```

The equivalent Weld program is similar. Weld programs are currently registered as strings, so we do some Python magic to splice in `number` into our string; the vector we operate on is similarly spliced in.

Note that the vector we operate over is the current value of `self.weldobj.weld_code`; this implies that we are running the `map` over what the current computation would return! We then assign this program to `self.weldobj.weld_code`, effectively updating the current computation.

Implementing the `__iadd__` function allows us to override the `+=` operator in Python.

In [153]:
def add(self, number):
    template = "map({0}, |e| e + {1})"
    self.weldobj.weld_code = template.format(
        self.weldobj.weld_code, str(number))
    return self

def __iadd__(self, other):
    return self.add(other)

### Implementing Vector Sum

Almost there! Now lets do the vector sum. Like before, we start with a template. The template looks a bit intimidating compared to our `map` function from before, but don't fret! If you look closely, all we're doing is running a `for` loop over some data.

The `merger`, `result`, and `merge` functions are operations over _builders_, which are special types in Weld used to capture parallelism. We won't go into much detail here, but basically the `merger[i64,+]` is doing a reduce on each element of the vector where the reduction function is `+`.

#### Materialization

The vector sum will return a single integer, but our class represents a vector! We can resolve this by _materializing a result_ in this function (_i.e.,_ actually evaluating the computation we've built up). In a fuller library, we wouldn't need to this here and we could track information about what type of object the current computation represents, but we won't do that here.

To evaluate an object, we simply call the `evaluate` method. We'll look at the encoders and decoders in the code below soon!

In [155]:
def vector_sum(self):
    template = "result(for({0}, merger[i64,+], |b,i,e| merge(b, i64(e))))"
    prev_code = self.weldobj.weld_code
    self.weldobj.weld_code = template.format(self.weldobj.weld_code)
    self.weldobj.decoder = ScalarDecoder()
    result = self.weldobj.evaluate(WeldLong(), verbose=False)
    self.weldobj.decoder = _decoder
    self.weldobj.weld_code = prev_code
    return result

Ignore this stuff! Just some Python hackery to add the above functions as methods...

In [7]:
setattr(HelloWeldVector, '__init__', classmethod(__init__))
setattr(HelloWeldVector, 'add', add)
setattr(HelloWeldVector, '__iadd__', __iadd__)
setattr(HelloWeldVector, 'sum', sum)

NameError: name '__init__' is not defined

### Encoders and Decoders

We saw several references to _encoders_ and _decoders_ above. What are those? Weld operates over its own internal data format; we need some way to tell Weld how to map data in Python (or in our case, NumPy) to a data format Weld can understand.

That's where encoders and decoders come in! An encoder takes data that will be passed into Weld and marshalls it into a Weld format. A decoder takes data returned by Weld and marshalls it into a format Python understands. _Often, but not always_, marshalling can be done with a simple pointer (instead of data) copy.

#### NumPy Encoders and Decoders

Weld declares an interface for writing encoders and decoders. Fortunately, since we are working with NumPy, encoders and decoders for NumPy arrays are already built into Weld.

In [159]:
from weld.encoders import NumpyArrayEncoder, NumpyArrayDecoder, ScalarDecoder

_encoder = NumpyArrayEncoder()
_decoder = NumpyArrayDecoder()

## Accelerating our Workload

We now have all the pieces we need! Let's copy our NumPy function from above, but make one modification; we'll wrap the NumPy array in a `HelloWeldVector`. We've tried our best to emulate NumPy's API, so the rest of the code looks the same!

In [166]:
def run_numpy_addition_with_weld(a):
    a = HelloWeldVector(a)
    for i in xrange(ITERATIONS):
        a += i
        
    print "Result:", a.sum()

The below function will time and compare the native NumPy function to the Weld function we just implemented.

In [168]:
def compare_against_numpy():
    a = np.zeros((SIZE), dtype='int32')

    start = time.time()
    run_numpy_addition(a)
    end = time.time()

    numpy_time = end - start

    # Because caches are funny things
    a = np.zeros((SIZE), dtype='int32')

    start = time.time()
    run_numpy_addition_with_weld(a)
    end = time.time()

    run_time = end - start

    print "Speedup:", numpy_time / run_time

In [171]:
compare_against_numpy()

Result: 418759311360
Result: 418759311360
Speedup: 7.4842164901


Not bad! By combining each operation into a single loop, we saw a > 5x speedup in performance!

Now, let's set the following environment variable, which will increase the _number of threads_ Weld is allowed to use. Because Weld is a _parallel language_, any loop we write in it can be automatically parallelized.

In [173]:
import os
os.environ["WELD_NUM_THREADS"] = "4"

compare_against_numpy()

Result: 418759311360
Result: 418759311360
Speedup: 9.60425773098


Not bad!

Note that the current version of Weld we're running is actually _missing_ many features discussed in the Weld paper. Namely:
 
 * Many optimizations are missing (e.g., common subexpression elimination, constant folding)
 * No vectorization
 
While some optimizations will be handled by LLVM, things like vectorization (coming soon!) can give even further speedups.

# Grizzly and Real Workloads

The workload we looked at above is actually quite simple, so maybe you're not yet impressed with the speedups Weld generated! We hear you! Grizzly is a subset of Pandas that we integrated with Weld. Grizzly allows us to port Pandas workloads over to use Weld, without changing the application!

Let's take Grizzly out for a spin!

First, let's import Pandas and Grizzly.

In [11]:
import pandas as pd
import grizzly.grizzly as gr

Now, let's write a function that loads some data. Note that Grizzly still depends on native Pandas for I/O.

In [12]:
def get_data_cleaning_data():
    na_values = ['NO CLUE', 'N/A', '0']
    requests = pd.read_csv('data/311-service-requests.csv', na_values=na_values, dtype={'Incident Zip': str})
    return requests

Now that we have loaded data in our CSV to a Pandas dataframe, it's time to play! The dataset we just loaded contains a bunch of zipcodes. But like most data our in the wild, it's noisy! :( Let's use Pandas to clean the data!

Some of the zipcodes contain more than 5 digits, so we're going to truncate every zipcode to its first 5 digits. In addition, some of the zipcodes are all-zero -- we're going to convert all those zipcodes to `nan`s. After these cleaning operations, we're going to print the unique zipcodes.

In [13]:
def run_data_cleaning_pandas(requests):
    requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)
    
    # Fix requests with 00000 zipcodes
    zero_zips = requests['Incident Zip'] == '00000'
    requests['Incident Zip'][zero_zips] = np.nan
    
    # Display unique zip codes.
    print 'Result:', requests['Incident Zip'].unique()

That wasn't too hard!

Now, let's write the same function using Grizzly! Grizzly shares the same API as Pandas, so we're only going to have to wrap the input dataframe in Grizzly's `DataFrameWeld` and we're off to the races!

In [14]:
def run_data_cleaning_grizzly(requests):
    requests = gr.DataFrameWeld(requests)
    requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)
    
    # Fix requests with 00000 zipcodes
    zero_zips = requests['Incident Zip'] == '00000'
    requests['Incident Zip'][zero_zips] = 'nan'
    
    # Display unique zip codes.
    print 'Result:', requests['Incident Zip'].unique().evaluate(verbose=False)

Now, let's load our data, and call the native Pandas function,

In [7]:
requests_orig = get_data_cleaning_data()

In [15]:
requests = requests_orig.copy()

start = time.time()
run_data_cleaning_pandas(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)

Result: ['11432' '11378' '10032' '10023' '10027' '11372' '11419' '11417' '10011'
 '11225' '11218' '10003' '10029' '10466' '11219' '10025' '10310' '11236'
 '10033' '11216' '10016' '10305' '10312' '10026' '10309' '10036' '11433'
 '11235' '11213' '11379' '11101' '10014' '11231' '11234' '10457' '10459'
 '10465' '11207' '10002' '10034' '11233' '10453' '10456' '10469' '11374'
 '11221' '11421' '11215' '10007' '10019' '11205' '11418' '11369' '11249'
 '10005' '10009' '11211' '11412' '10458' '11229' '10065' '10030' '11222'
 '10024' '10013' '11420' '11365' '10012' '11214' '11212' '10022' '11232'
 '11040' '11226' '10281' '11102' '11208' '10001' '10472' '11414' '11223'
 '10040' '11220' '11373' '11203' '11691' '11356' '10017' '10452' '10280'
 '11217' '10031' '11201' '11358' '10128' '11423' '10039' '10010' '11209'
 '10021' '10037' '11413' '11375' '11238' '10473' '11103' '11354' '11361'
 '11106' '11385' '10463' '10467' '11204' '11237' '11377' '11364' '11434'
 '11435' '11210' '11228' '11368' '11694' '1

We can do the same thing with Grizzly now. First, let's run Grizzly using just the one thread,

In [ ]:
import os
os.environ["WELD_NUM_THREADS"] = "1"

requests = requests_orig.copy()

start = time.time()
run_data_cleaning_grizzly(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)

Well, that's kinda cool!

What if we increase the number of threads?

In [ ]:
import os
os.environ["WELD_NUM_THREADS"] = "4"

start = time.time()
run_data_cleaning_grizzly(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)

Not bad at all!

With almost zero change to the application, we were able to get a 5x speedup! [fix this number]